# Chat with prompty

**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using prompty and visualize the trace of your application.
- Understand how to handle chat conversation using prompty
- batch run prompty against multi lines of data.


## 0. Install dependent packages

In [ ]:
%%capture --no-stderr
%pip install promptflow-core

## 1. Prompty

Prompty is a file with .prompty extension for developing prompt template. 
The prompty asset is a markdown file with a modified front matter. 
The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [ ]:
with open("chat.prompty") as fin:
    print(fin.read())

Note: before running below cell, please configure required environment variable `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT` by create an `.env` file. Please refer to [.env.example](../.env.example) as an template.

In [ ]:
from promptflow.core import Flow

# load prompty as a flow
f = Flow.load("chat.prompty")
# execute the flow as function
question = "What is the capital of France?"
result = f(firstName="John", lastName="Doe", question=question)
result

### Visualize trace by using start_trace

In [ ]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Re-run below cell will collect a trace in trace UI.

In [ ]:
# rerun the function, which will be recorded in the trace
result = f(firstName="John", lastName="Doe", question=question)
result

### Eval the result 

In this example, we will use a prompt that determines whether a chat conversation contains an apology from the assistant.

In [ ]:
eval_prompty = "../eval-apology/apology.prompty"

with open(eval_prompty) as fin:
    print(fin.read())

In [ ]:
# load prompty as a flow
eval_flow = Flow.load(eval_prompty)
# execute the flow as function
result = eval_flow(question=question, answer=result["answer"], messages=[])
result

## 2. Batch run with multi-line data


In [ ]:
%%capture --no-stderr
# batch run requires promptflow-devkit package
%pip install promptflow-devkit

In [ ]:
from promptflow.client import PFClient

pf = PFClient()

In [ ]:
flow = "chat.prompty"  # path to the prompty file
data = "./data.jsonl"  # path to the data file

# create run with the flow and data
base_run = pf.run(
    flow=flow,
    data=data,
    stream=True,
)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3. Evaluate your prompty
Then you can use an evaluation prompty to evaluate your prompty.

### Run evaluation on the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.

In [ ]:
eval_run = pf.run(
    flow=eval_prompty,
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "messages": "${data.chat_history}",
        "question": "${data.question}",
        "answer": "${run.outputs.answer}",
    },
    stream=True,
)

In [ ]:
details = pf.get_details(eval_run)
details.head(10)

# Next Steps

By now you've successfully run your first prompt flow and even did evaluation on it. That's great!

You can check out more examples:
- [Basic Chat](../chat-basic/README.md): demonstrates how to create a chatbot that can remember previous interactions and use the conversation history to generate next message.